# Propose centroid replacing the masters with BERT siamese

In [1]:
# import tensorflow as tf
import keras
# from tensorflow.python import keras
import os

Using TensorFlow backend.


In [2]:
# Optimize the use of GPUs
# https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras
# https://keras.io/getting-started/faq/#how-can-i-run-a-keras-model-on-multiple-gpus
# https://stackoverflow.com/questions/56316451/how-to-use-specific-gpus-in-keras-for-multi-gpu-training

In [3]:
from __future__ import print_function, division

In [4]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [5]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras import optimizers

In [6]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [7]:
MAX_SEQUENCE_LENGTH_T = 20 # 20
MAX_SEQUENCE_LENGTH_D = 20 # 80
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'deepQL_weights_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
# !wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip -o uncased_L-12_H-768_A-12.zip

In [10]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [11]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [12]:
"Total vocabulary: {}".format(len(token_dict))

'Total vocabulary: 30522'

In [13]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 
                    token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [14]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [15]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [16]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 36.5 s, sys: 9.52 s, total: 46 s
Wall time: 1min 17s


#### Hashing bugs by buckets

In [17]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [18]:
%%time

experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids
CPU times: user 2min 36s, sys: 15.9 ms, total: 2min 36s
Wall time: 2min 36s


#### Recovery bug ids from train

In [19]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [20]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '1\n',
 'bug_status': '2\n',
 'component': '475\n',
 'creation_ts': '2005-07-15 18:39:00 -0400',
 'delta_ts': '2013-02-20 16:06:43 -0500',
 'description': '[CLS] the sql editor should use . sql as the default extension . this will allows the sql editor to be used the next time the user tries to open the script . [SEP]',
 'description_bert': '[CLS] the sql editor should use . sql as the default extension . this will allows the sql editor to be used the next time the user tries to open the script . [SEP]',
 'description_word': array([  101,  1996, 29296,  3559,  2323,  2224,  1012, 29296,  2004,
         1996, 12398,  5331,  1012,  2023,  2097,  4473,  1996, 29296,
         3559,   102]),
 'description_word_bert': [101,
  1996,
  29296,
  3559,
  2323,
  2224,
  1012,
  29296,
  2004,
  1996,
  12398,
  5331,
  1012,
  2023,
  2097,
  4473,
  1996,
  29296,
  3559,
  2000,
  2022,
  2109,
  1996,
  2279,
  2051,
  1996,
  5310,
  5363,
  2000,
  2330,
  1996,
  5896,
  1

### Generating the batch test

In [21]:
"Train ", len(baseline.dup_sets_train)

('Train ', 34882)

In [22]:
bug_idx = bug_train_ids[0]
vector = baseline.bug_set[bug_idx]['textual_word']
annoy_train = AnnoyIndex(vector.shape[0])
for bug_id in bug_train_ids:
    annoy_train.add_item(bug_id, baseline.bug_set[bug_id]['textual_word'])
annoy_train.build(10) # 10 trees
"Indexed all train"

'Indexed all train'

### Train ids

In [23]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [24]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, \
                            valid_master_sample, valid_master_neg, valid_sim = experiment.batch_iterator_bert(None, baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1, 
                                                                                              issues_by_buckets, INCLUDE_MASTER=True)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

CPU times: user 66.3 ms, sys: 0 ns, total: 66.3 ms
Wall time: 65.6 ms


In [25]:
valid_input_sample['title']['token'].shape, valid_input_sample['description']['token'].shape, \
    valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 1682), (128,))

### Validar entrada

In [26]:
# %%time 

# baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5, batch_iterator, issues_by_buckets)

## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [27]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### BERT

https://github.com/CyberZHG/keras-bert

In [28]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
#     outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [29]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    #layer = GRU(100, activation='tanh')(layer)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [435]:
from keras import backend as K
from keras.layers import Layer

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=True)
    #return K.mean(distance, axis=-1, keepdims=False)

def _triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def triplet_loss(vects):
    pos = vects[0]
    neg = vects[1]
    margin = K.constant(1.0)
    return K.maximum(0.0, margin - pos + neg)
    #return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

class QuintetWeights(Layer):
 
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(QuintetWeights, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.W= self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.built = False 
 
    def call(self, x):
        return K.dot(x, self.W)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

class Triplet(Layer):
 
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(Triplet, self).__init__(**kwargs)
 
    def build(self, input_shape):
        pass
 
    def call(self, x):
        pos, neg  = x
        margin = K.constant(1.0)
        return K.maximum(0.0, margin - pos + neg)
    
def custom_loss(y_true, y_pred):
    return y_pred[0][0]

def triplet_bug(y_true, y_pred):
    return y_pred[0][1]
def triplet_anchor(y_true, y_pred):
    return y_pred[0][2]
def triplet_pos(y_true, y_pred):
    return y_pred[0][3]
def triplet_neg(y_true, y_pred):
    return y_pred[0][4]

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

### Propose

In [365]:
from keras.layers import concatenate, Add, Lambda, Average, Maximum, Subtract, Average, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
    
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [366]:
def siamese_model_centroid(sequence_length, name):
    
    # Bug Centroid Feature
    bug_centroid = Input(shape = (sequence_length, ), name = 'bug_centroid_feature_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_centroid], outputs=[bug_centroid], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [436]:
from keras.layers import Average, Reshape, Add
from keras_radam import RAdam
from keras_bert import AdamWarmup, calc_train_steps

def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                             master_anchor, master_negative, master_positive, 
                         NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    inputs.append(master_anchor.input)
    inputs.append(master_positive.input)
    inputs.append(master_negative.input)
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    master_anchor = master_anchor.output
    master_negative = master_negative.output
    master_positive = master_positive.output
    
    # Distance bugs
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance')([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance')([encoded_anchor, encoded_negative])
    
    # Distance masters anchor
    master_anchor_positive_d = Lambda(cosine_distance, name='pos_master_cosine_distance')([encoded_anchor, master_positive])
    master_anchor_negative_d = Lambda(cosine_distance, name='neg_master_cosine_distance')([encoded_anchor, master_negative])
    
    # Distance master positive
    master_pos_positive_d = Lambda(cosine_distance, name='pos_master_pos_cosine_distance')([encoded_positive, master_positive])
    master_pos_negative_d = Lambda(cosine_distance, name='neg_master_pos_cosine_distance')([encoded_positive, master_negative])
    
    # Distance master negative
    master_neg_positive_d = Lambda(cosine_distance, name='pos_master_neg_cosine_distance')([encoded_negative, master_negative])
    master_neg_negative_d = Lambda(cosine_distance, name='neg_master_neg_cosine_distance')([encoded_negative, master_positive])
     
#     output_bug = Triplet(1)([positive_d, negative_d])
#     output_bug = QuintetWeights((1,2))(output_bug)
    output_bug = Lambda(triplet_loss, name='triplet_pos_neg')([positive_d, negative_d])
    output_master = Lambda(triplet_loss, name='triplet_anchor_centroid')([master_anchor_positive_d, master_anchor_negative_d])
    output_master_pos = Lambda(triplet_loss, name='triplet_pos_centroid')([master_pos_positive_d, master_pos_negative_d])
    output_master_neg = Lambda(triplet_loss, name='triplet_neg_centroid')([master_neg_positive_d, master_neg_negative_d])
    
    # Weights for each loss
    output_bug = Reshape((1, ))(output_bug)
    output_anchor = Reshape((1, ))(output_master)
    output_pos = Reshape((1, ))(output_master_pos)
    output_neg = Reshape((1, ))(output_master_neg)
    
    sum_all_losses = Add()([output_bug, output_master, output_master_pos, output_master_neg])
    output_bug_w = Dense(1, input_shape=(1,), use_bias=False)(output_bug)
    output_master_w = Dense(1, input_shape=(1,), use_bias=False)(output_anchor)
    output_master_pos_w = Dense(1, input_shape=(1,), use_bias=False)(output_pos)
    output_master_neg_w = Dense(1, input_shape=(1,), use_bias=False)(output_neg)
    sum_all_loss_w = Add()([output_bug_w, output_master_w, output_master_pos_w, output_master_neg_w])
    # Weighted Median Loss
    output_median = Lambda(
            lambda inputs: inputs[0] / inputs[1], 
            name = 'Weighted_Median_Loss')([sum_all_loss_w, sum_all_losses])
    
    output = concatenate([output_median, output_bug, output_anchor, output_pos, output_neg])

    similarity_model = Model(inputs = inputs, outputs = [output], name = 'Similarity_Model')

    # setup the optimization process 
    similarity_model.compile(optimizer='adam',
                             metrics=[triplet_bug, triplet_anchor, triplet_pos, triplet_neg],
                             loss=custom_loss)

    # metrics=[triplet_bug, triplet_anchor, triplet_pos, triplet_neg],
    return similarity_model

In [438]:
%%time
import os

print("Batch size ", batch_size)

# Inspired on https://'pastebin.com/TaGFdcBA
keras.backend.clear_session()

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
# title_feature_model = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
#desc_feature_model = cnn_model(desc_embedding_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')
# Master model
embed_size = K.int_shape(title_feature_model.get_output_at(0))[1] + K.int_shape(desc_feature_model.get_output_at(0))[1] + K.int_shape(categorical_feature_model.get_output_at(0))[1] 

master_anchor = siamese_model_centroid(embed_size, 'master_anchor')
master_pos = siamese_model_centroid(embed_size, 'master_pos')
master_negative = siamese_model_centroid(embed_size, 'master_neg')

NUMBER_OF_INSTANCES = len(baseline.dup_sets_train)
BATCH_SIZE = batch_size
EPOCHS = epochs

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, 
                                            master_anchor, master_negative, master_pos,
                                            NUMBER_OF_INSTANCES, BATCH_SIZE, EPOCHS, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, train_master_input, train_master_neg, \
            train_sim = experiment.batch_iterator_bert(encoded_anchor, baseline.train_data, baseline.dup_sets_train, \
                                                       bug_train_ids, 
                                                       batch_size, 1, 
                                                       issues_by_buckets, 
                                                       TRIPLET_HARD=True, USE_CENTROID=True)
    
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], train_input_sample['description']['token'], train_input_sample['description']['segment'], train_input_sample['info'],
                   train_input_pos['title']['token'], train_input_pos['title']['segment'], train_input_pos['description']['token'], train_input_pos['description']['segment'], train_input_pos['info'], 
                   train_input_neg['title']['token'], train_input_neg['title']['segment'], train_input_neg['description']['token'], train_input_neg['description']['segment'], train_input_neg['info'],
                   train_master_input['centroid_embed'],
                   train_master_input['centroid_embed'],
                   train_master_neg['centroid_embed']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, TL: {:.2f}, TL_A: {:.2f}, TL_P: {:.2f}, TL_N: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h[1], h[2], h[3], h[4], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, TL: {:.2f}, TL_A: {:.2f}, TL_P: {:.2f}, TL_N: {:.2f}".format(epoch+1,h[0], h[1], h[2], h[3], h[4]))
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}s, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

Batch size  64
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
______________________________________________________________________________________________

Epoch: 1 Loss: 0.93, TL: 1.00, TL_A: 0.96, TL_P: 0.95, TL_N: 0.97
Epoch: 2 Loss: 0.93, TL: 1.01, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 3 Loss: 0.93, TL: 1.01, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 4 Loss: 0.93, TL: 1.01, TL_A: 0.97, TL_P: 0.96, TL_N: 0.97
Epoch: 5 Loss: 0.93, TL: 1.02, TL_A: 0.97, TL_P: 0.97, TL_N: 0.98
Epoch: 6 Loss: 0.93, TL: 1.02, TL_A: 0.98, TL_P: 0.97, TL_N: 0.98
Epoch: 7 Loss: 0.92, TL: 1.01, TL_A: 0.97, TL_P: 0.97, TL_N: 0.98
Epoch: 8 Loss: 0.92, TL: 1.02, TL_A: 0.98, TL_P: 0.97, TL_N: 0.98
Epoch: 9 Loss: 0.92, TL: 1.02, TL_A: 0.98, TL_P: 0.97, TL_N: 0.98
Epoch: 10 Loss: 0.92, TL: 1.01, TL_A: 0.98, TL_P: 0.98, TL_N: 0.98
Epoch: 11 Loss: 0.92, TL: 1.01, TL_A: 0.98, TL_P: 0.98, TL_N: 0.98
Epoch: 12 Loss: 0.92, TL: 1.00, TL_A: 0.98, TL_P: 0.98, TL_N: 0.98
Epoch: 13 Loss: 0.92, TL: 1.01, TL_A: 0.99, TL_P: 0.98, TL_N: 0.99
Epoch: 14 Loss: 0.92, TL: 1.01, TL_A: 0.99, TL_P: 0.98, TL_N: 0.99
Epoch: 15 Loss: 0.91, TL: 1.01, TL_A: 0.99, TL_P: 0.98, TL_N: 0.99
Epoc

In [439]:
recall

0.37

In [440]:
_[:20]

['327681:324658|420526:0.955991443246603,422622:0.9559594579041004,411055:0.9557418264448643,418146:0.9555703476071358,376670:0.9554064050316811,403466:0.9549705162644386,419892:0.9549618251621723,403948:0.9549268037080765,418858:0.954805564135313,417798:0.9546240381896496,391963:0.9545932747423649,335784:0.9545860812067986,369596:0.9543934501707554,336922:0.9543560147285461,381119:0.9543355815112591,368850:0.9542971849441528,329597:0.9542873874306679,418838:0.9542631395161152,422095:0.9542536735534668,381298:0.9542535804212093,419581:0.9542295597493649,387326:0.9542203731834888,419879:0.9541943408548832,381296:0.9541781470179558,419883:0.9541210383176804,381138:0.9540407098829746,419880:0.9540344476699829,387399:0.9540259204804897,385830:0.954003244638443',
 '360457:362252|364552:0.980149183422327,353496:0.975926611572504,353222:0.9745746776461601,351842:0.9709462597966194,355947:0.9703360050916672,368089:0.970120295882225,367431:0.96998062543571,348343:0.9694935996085405,343568:0.969

In [441]:
# recall, exported_rank, debug = experiment.evaluate_validation_test(experiment, retrieval, verbose, 
#                                                         encoded_anchor, issues_by_buckets, evaluate_validation_test)
# test_vectorized, queries_test_vectorized, annoy, X_test, distance_test, indices_test = debug
# "recall@25 last epoch:", recall

### Retrieval evaluation

In [442]:
print("Total of queries:", len(retrieval.test))

Total of queries: 4641


#### Getting the model trained

In [443]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'deepQL_weights_100_feature_100epochs_64batch(eclipse)'

In [444]:
model = encoded_anchor
# model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

In [445]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [446]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, 
                                                                   bug_train_ids, method='bert')

In [447]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/eclipse/exported_rank_deepQL_weights_100.txt'

In [448]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [449]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.24,
 '2 - recall_at_10': 0.29,
 '3 - recall_at_15': 0.33,
 '4 - recall_at_20': 0.35,
 '5 - recall_at_25': 0.37}